In [1]:
from sou.reasoning.run_reasoning import run_reasoning_loop
from sou.reasoning.config import ReasoningSettings

settings = ReasoningSettings(
    model_name="groq/meta-llama/llama-4-scout-17b-16e-instruct",
)


/Users/yulin/anaconda3/envs/agentic_reasoning1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/yulin/anaconda3/envs/agentic_reasoning1/lib/python3.11/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [2]:
prompt = "What is llama4? Do you think its score in benchmark is like presented in the technical report of meta team or is it maybe a scam? "
final_sequence = run_reasoning_loop(prompt, reasoning_settings=settings)

19:21:23 - LiteLLM:INFO: utils.py:2894 - 
LiteLLM completion() model= llama3-70b-8192; provider = groq
INFO:LiteLLM:
LiteLLM completion() model= llama3-70b-8192; provider = groq


===========token count 49

You are an assistant for summarization tasks. Use the following pieces of retrieved context to summarize the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.



INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
19:21:27 - LiteLLM:INFO: utils.py:1084 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
Token indices sequence length is longer than the specified maximum sequence length for this model (1171 > 1024). Running this sequence through the model will result in indexing errors
19:21:27 - LiteLLM:INFO: utils.py:2894 - 
LiteLLM completion() model= meta-llama/llama-4-scout-17b-16e-instruct; provider = groq
INFO:LiteLLM:
LiteLLM completion() model= meta-llama/llama-4-scout-17b-16e-instruct; provider = groq


===========token count 1171
You are a reasoning assistant with the ability to perform web searches and execute code to help you answer the user's question accurately. You have special tools:

- To perform a search: write <begin_search_query> your query here </end_search_query>.
Then, the system will search and analyze relevant web pages, then provide you with helpful information in the format <begin_search_result> ...search results... </end_search_result>.
Make sure your each search query is clear and concise.

- To perform calculations or data processing, you can propose a code task using: <begin_code_query> your code query here </end_code_query>.
The system will write the code and execute the code then return the results in the format <begin_code_result> ...execution results... </end_code_result>.
Make sure your each code query is self-contained and does not require any external information.

- To access your reasoning memory, you can query the automatically generated mind map of you

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
19:21:29 - LiteLLM:INFO: utils.py:1084 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler


...

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
19:21:39 - LiteLLM:INFO: utils.py:1084 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler


=======Output: Based on the search results and my prior knowledge, here is a summary of the information about Llama4:

Llama4 is a series of large language models developed by Meta, with recent releases including Llama4 Scout, Llama4 Maverick, and Llama4 Behemoth. These models have achieved state-of-the-art results in various benchmarks.

However, there have been allegations that Meta manipulated the Llama4 models to achieve better benchmark scores while hiding their limitations. Specifically, it was claimed that Meta trained the models on test sets, which is not a standard practice.

Meta has denied these allegations, stating that they are "simply not true" and that they would never train models on test sets. They attributed the mixed performance reports to implementation stability rather than flaws in the training process.

The controversy highlights a challenge in the AI industry, where benchmark scores often fail to reflect real-world capabilities. Researchers have noted discrepanc

19:21:39 - LiteLLM:INFO: utils.py:2894 - 
LiteLLM completion() model= meta-llama/llama-4-scout-17b-16e-instruct; provider = groq
INFO:LiteLLM:
LiteLLM completion() model= meta-llama/llama-4-scout-17b-16e-instruct; provider = groq


===========token count 665

    Given the following context: Based on the search results and my prior knowledge, here is a summary of the information about Llama4:

Llama4 is a series of large language models developed by Meta, with recent releases including Llama4 Scout, Llama4 Maverick, and Llama4 Behemoth. These models have achieved state-of-the-art results in various benchmarks.

However, there have been allegations that Meta manipulated the Llama4 models to achieve better benchmark scores while hiding their limitations. Specifically, it was claimed that Meta trained the models on test sets, which is not a standard practice.

Meta has denied these allegations, stating that they are "simply not true" and that they would never train models on test sets. They attributed the mixed performance reports to implementation stability rather than flaws in the training process.

The controversy highlights a challenge in the AI industry, where benchmark scores often fail to reflect real-world c

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
19:21:40 - LiteLLM:INFO: utils.py:1084 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler


In [3]:
final_sequence.final_result

"Based on the provided context, here's an answer to your question:\n\n**What is Llama4?**\nLlama4 is a series of large language models developed by Meta, with recent releases including Llama4 Scout, Llama4 Maverick, and Llama4 Behemoth. These models have achieved state-of-the-art results in various benchmarks.\n\n**Is Llama4's benchmark score legitimate or a scam?**\nThere are concerns about the legitimacy of Llama4's benchmark scores. While Meta has denied allegations that they manipulated the models to achieve better benchmark scores, there have been claims that they might have trained the models on test sets, which is not a standard practice. This has raised questions about the validity of their benchmark scores.\n\nIt's essential to approach this information with caution and consider multiple sources before making a judgment. While Llama4 appears to be a legitimate model developed by Meta, its benchmark scores and performance require further evaluation.\n\nThe code provided to asse

In [4]:
for i in final_sequence.history:
    print(i)

To address your question about Llama4, I will first perform a web search to gather information about it.

Assistant:
<begin_search_query>Llama4 benchmark score technical report meta team</end_search_query>

<begin_search_result>
The search results indicate that there is limited information available about Llama4. However, I found some information about Llama, which is a series of large language models developed by Meta. The most recent model in this series is likely Llama3, but there are mentions of future models. 

One of the results mentions a technical report by Meta, which discusses the performance of their Llama models in various benchmarks. However, there is no direct mention of Llama4 or its benchmark scores.

Another result suggests that there might be some confusion or misinformation about Llama4, with some sources referring to it as a potential scam or a model that does not exist.

To verify the authenticity and benchmark scores of Llama4, I would need to look into the techni

In [5]:
final_sequence.output

'Based on the search results and my prior knowledge, here is a summary of the information about Llama4:\n\nLlama4 is a series of large language models developed by Meta, with recent releases including Llama4 Scout, Llama4 Maverick, and Llama4 Behemoth. These models have achieved state-of-the-art results in various benchmarks.\n\nHowever, there have been allegations that Meta manipulated the Llama4 models to achieve better benchmark scores while hiding their limitations. Specifically, it was claimed that Meta trained the models on test sets, which is not a standard practice.\n\nMeta has denied these allegations, stating that they are "simply not true" and that they would never train models on test sets. They attributed the mixed performance reports to implementation stability rather than flaws in the training process.\n\nThe controversy highlights a challenge in the AI industry, where benchmark scores often fail to reflect real-world capabilities. Researchers have noted discrepancies be